2015 CAASPP test results.


* [Source Website](http://caaspp.cde.ca.gov/sb2015/ResearchFileList)
* [Data Dictionary](http://caaspp.cde.ca.gov/sb2015/research_fixfileformat)

The county code for San Diego is 37.

In [265]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import pandas as pd
df = pd.read_csv('../ca2015_all_csv_v1.txt', na_values='*'  )
ent = pd.read_csv('../ca2015entities_csv.txt')

df = df[df['Test Id'] == 1]

def f(x):
    return "{:02d}{:05d}{:07d}".format(x['County Code'],x['District Code'],x['School Code'])
   
df['cds'] = df.apply(f, axis=1)
ent['cds'] = ent.apply(f, axis=1)
ent.set_index('cds',inplace = True)

county_names = ent['County Name']
school_names = ent['School Name']
district_names = ent[['District Name', 'County Name']]

race_eth_subgroups = [200,202,203,204,206]+[220, 222, 223, 224, 226]
race_eth_names = ['BlackED', 'AsianED', 'FilipinoED' , 'LatinoED', 'WhiteED', 
                    'BlackP', 'AsianP', 'FilipinoP' , 'LatinoP', 'WhiteP']


subgroups = pd.read_csv('../Subgroups.txt', header=None, names = ['subgroup_id','subgroup_name','category'],
                       index_col = 0)
subgroups['title'] = subgroups.apply(lambda row: row['category']+"; "+row['subgroup_name'], axis=1)
subgroup_names = subgroups.to_dict()['title']

In [299]:
subgroup_names = subgroups.to_dict()['title']

In [306]:

def geo_decomp(grade):
    def subset(grade, df):
        dfs = df[['cds', 'County Code', 'Subgroup ID', 'Total Tested At Subgroup Level', 'Grade','Mean Scale Score', 
             'Percentage Standard Met and Above']]
        dfs.columns = [u'cds', 'county',  u'subgroup', u'subgroup_tested', u'grade', u'mss', u'pct_msa']

        return dfs[dfs.grade == grade]

    schools = subset(grade, df[df['School Code'] != 0])
    districts =  subset(grade, df[(df['District Code'] != 0) & (df['School Code'] == 0)])
    counties =  subset(grade, df[ (df['County Code'] != 0) & (df['District Code'] == 0) & (df['School Code'] == 0)])
    states =  subset(grade, df[ (df['County Code'] == 0)])

    counties_p = pd.pivot_table(counties,index='cds', columns='subgroup', values='pct_msa')
    districts_p = pd.pivot_table(districts[district.pct_msa != 0],index='cds', columns='subgroup', values='pct_msa')
    schools_p = pd.pivot_table(schools[schools.pct_msa != 0],index='cds', columns='subgroup', values='pct_msa')
    
    return (state, counties, counties_p, districts, districts_p, schools, schools_p)
   
# Join Names!
# .join(district_names,how='left', rsuffix='_r')
   
def munge_sg_cols(df):
    df.columns = [ subgroup_names.get(e,e).strip().strip('"') for e in df.columns ]
    return df
    
def district_subgroups(grade, subgroup_ids, munge_cols = True):
    
    (state, counties, counties_p, districts, districts_p, schools, schools_p) = geo_decomp(grade)
    
    ed = districts[districts.subgroup.isin(subgroup_ids) & (districts.county == 37)]
    
    df =  pd.pivot_table(ed,index='cds', columns='subgroup', values='pct_msa')\
    .join(district_names,how='left', rsuffix='_r').sort(subgroup_ids[0], ascending=False)
    
    #df =  pd.pivot_table(ed[districts.pct_msa != 0],index='cds', columns='subgroup', values='pct_msa')\
    #.join(district_names,how='left', rsuffix='_r').sort(subgroup_ids[0], ascending=False)
    
    if munge_cols:
        munge_sg_cols(df)
  
    
    return df

def district_difs(grade, sg_id1, sg_id2):
    
    df = district_subgroups(grade, (sg_id1, sg_id2), munge_cols = False)
    
    df['diff'] = df[sg_id1] - df[sg_id2]
    
    munge_sg_cols(df)
    
    return df.sort('diff',ascending = True)
    

## State Average By Subgroup

In [284]:
pd.set_option('display.max_colwidth', 120)
sa = pd.pivot_table(state,index='cds', columns='subgroup', values='pct_msa').T.join(subgroups['title'])
sa.columns = ['pct_msa', 'Subgroup Title']
sa.sort('pct_msa', ascending = True)

,pct_msa,Subgroup Title
subgroup,,
222,76,Ethnicity for Not Economically Disadvantaged; Asian
94,69,Parent Education; Graduate School/Post Graduate
7,67,English-Language Fluency; Initially-Fluent English Proficient (I-FEP)
223,65,Ethnicity for Not Economically Disadvantaged; Filipino
76,65,Ethnicity; Asian
226,64,Ethnicity for Not Economically Disadvantaged; White
227,63,Ethnicity for Not Economically Disadvantaged; Ethnicity -- Two or More Races
111,59,Economic Status; Not Economically Disadvantaged
8,59,English-Language Fluency; Reclassified-Fluent English Proficient (R-FEP)


## County Average

In [198]:
county[(county.county == 37) & ( county.subgroup.isin((31,111)))]

,cds,county,subgroup,subgroup_tested,grade,mss,pct_msa
2034275,37000000000000,37,31,152522,3,2387.2,30
2034499,37000000000000,37,111,140374,3,2458.3,64


In [199]:
county_p[[31,111]].describe().T

,count,mean,std,min,25%,50%,75%,max
31,57,24.052632,5.214749,15,21,23,26,42
111,57,53.263158,10.024125,30,48,54,60,75


## Disadvantage, by San Diego County District 

Ranked by MSA for non-disadvantaged.

*Note* When limited to 3rd grade, why does list include Sweetwater Union High? 


In [307]:
district_difs(3, 204, 206)

,Ethnicity for Economically Disadvantaged; Hispanic or Latino,Ethnicity for Economically Disadvantaged; White,District Name,County Name,diff
cds,,,,,
37683950000000,17,50,South Bay Union School District,San Diego,-33
37735510000000,34,65,Carlsbad Unified,San Diego,-31
37737910000000,24,55,San Marcos Unified,San Diego,-31
37683380000000,28,56,San Diego Unified,San Diego,-28
37683040000000,21,44,Ramona City Unified,San Diego,-23
37682960000000,25,46,Poway Unified,San Diego,-21
37682130000000,16,36,Mountain Empire Unified,San Diego,-20
37684520000000,21,39,Vista Unified,San Diego,-18
37735690000000,15,33,Oceanside Unified,San Diego,-18


*note* Explore Borrego springs. They have 68% ED pass, but very low numbers for male and female. 

In [308]:
district_difs(3, 31,111)

,Economic Status; Economically Disadvantaged,Economic Status; Not Economically Disadvantaged,District Name,County Name,diff
cds,,,,,
37680800000000,18,73,Encinitas Union Elementary School Distri,San Diego,-55
37683870000000,45,88,Solana Beach Elementary,San Diego,-43
37737910000000,30,72,San Marcos Unified,San Diego,-42
37682960000000,34,73,Poway Unified,San Diego,-39
37735510000000,44,79,Carlsbad Unified,San Diego,-35
37683380000000,35,70,San Diego Unified,San Diego,-35
37679910000000,25,59,Cajon Valley Union,San Diego,-34
37680070000000,36,68,Cardiff Elementary School District,San Diego,-32
37735690000000,19,51,Oceanside Unified,San Diego,-32


In [311]:
district_difs(3, 200,202)

,Ethnicity for Economically Disadvantaged; Black or African American,Ethnicity for Economically Disadvantaged; Asian,District Name,County Name,diff
cds,,,,,
37680230000000,24,72,Chula Vista Elementary,San Diego,-48
37681970000000,19,59,La Mesa-Spring Valley,San Diego,-40
37683380000000,26,49,San Diego Unified,San Diego,-23
37682960000000,19,40,Poway Unified,San Diego,-21
37679910000000,21,38,Cajon Valley Union,San Diego,-17
37682050000000,21,33,Lemon Grove School District,San Diego,-12
37682210000000,36,39,National Elementary School District,San Diego,-3
37681140000000,36,NaN,Fallbrook Union Elementary,San Diego,NaN
37683950000000,32,NaN,South Bay Union School District,San Diego,NaN


In [312]:
district_difs(3, 91,93)

,Parent Education; High School Graduate,Parent Education; College Graduate,District Name,County Name,diff
cds,,,,,
37735510000000,31,81,Carlsbad Unified,San Diego,-50
37683870000000,38,84,Solana Beach Elementary,San Diego,-46
37737910000000,30,70,San Marcos Unified,San Diego,-40
37756140000000,26,64,Valley Center-Pauma Unified,San Diego,-38
37683040000000,23,60,Ramona City Unified,San Diego,-37
37682960000000,35,67,Poway Unified,San Diego,-32
37680800000000,36,67,Encinitas Union Elementary School Distri,San Diego,-31
37683380000000,33,63,San Diego Unified,San Diego,-30
37684520000000,25,51,Vista Unified,San Diego,-26


In [313]:
district_difs(3, 3,4)

,Gender; Males,Gender; Females,District Name,County Name,diff
cds,,,,,
37682210000000,22,38,National Elementary School District,San Diego,-16
37681970000000,26,41,La Mesa-Spring Valley,San Diego,-15
37768510000000,44,59,Bonsall Unified,San Diego,-15
37681140000000,38,52,Fallbrook Union Elementary,San Diego,-14
37683120000000,73,86,Rancho Santa Fe Elementary School Distri,San Diego,-13
37683790000000,26,38,San Ysidro Elementary,San Diego,-12
37680070000000,59,71,Cardiff Elementary School District,San Diego,-12
37680230000000,43,55,Chula Vista Elementary,San Diego,-12
37735690000000,22,33,Oceanside Unified,San Diego,-11
